<a href="https://colab.research.google.com/github/akshaygopan/RLVECN_learning/blob/main/RLVECN_Node_classification_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip generic_datasets.zip

In [ ]:
pip install ampligraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.0/204.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.5/575.5 kB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 531.9/531.9 kB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 6.5 MB/s

In [ ]:
import os

import tensorflow as tf
from keras import backend as K
from numpy.random import seed
seed(1)
tf.compat.v1.set_random_seed(3)

# load required modules
from datetime import datetime
import pandas as pd
import numpy as np
import json, csv, math, time
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D, Conv2D, MaxPooling2D
from keras import initializers, losses, metrics, optimizers
from keras.layers import BatchNormalization
from keras.layers import Embedding
from keras.utils import to_categorical
from keras.regularizers import l1_l2
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
import matplotlib.pyplot as plt
from statistics import mean
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.under_sampling import ClusterCentroids, RandomUnderSampler, NearMiss, TomekLinks
from ampligraph.latent_features import ScoringBasedEmbeddingModel



In [ ]:
REMOTE_URL = "https://snap.stanford.edu/data/gemsec_deezer_dataset.tar.gz"
LOCAL_PATH = "generic_datasets/graph_embeddings"
FILE_NAME = "gemsec_deezer_dataset.tar.gz"
graph_data = ["Facebook-Page2Page", "PubMed-Diabetes", "Terrorists-Relation"]  # ["Cora", "CiteSeer", "Facebook-Page2Page", "PubMed-Diabetes", "Terrorists-Relation", "Zachary-Karate", "Internet-Industry-Partnerships"]  # [sparse, dense]

for i in range(len(graph_data)):
    graph_fname = "/../" + graph_data[i] + "/" + graph_data[i]
    if (not os.path.isfile(LOCAL_PATH + graph_fname + ".labels")) or (not os.path.isfile(LOCAL_PATH + graph_fname + ".edges")):
      print("inside if working")
    FILE_NAME1 = LOCAL_PATH + graph_fname + ".labels"
    FILE_NAME2 = LOCAL_PATH + graph_fname + ".edges"
    df_cln1 = pd.read_csv (FILE_NAME1, sep="\s", header=0, index_col=None)
    df_cln1 = df_cln1.astype('int32')
    node_list = list(df_cln1['node_id'])
    X = df_cln1.values[:,0]  # "values()" method returns a NUMPY array wrt dataframes
    y = df_cln1.values[:,-1]  # "values()" method returns a NUMPY array wrt dataframes
    df_cln2 = pd.read_csv(FILE_NAME2, sep="\s", header=None, index_col=None)
    df_cln2 = df_cln2.astype('int32')
    df_cln2.columns = ['source_id', 'dest_id']


<ipython-input-15-a638c202653f>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_cln1 = pd.read_csv (FILE_NAME1, sep="\s", header=0, index_col=None)
<ipython-input-15-a638c202653f>:17: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_cln2 = pd.read_csv(FILE_NAME2, sep="\s", header=None, index_col=None)
<ipython-input-15-a638c202653f>:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_cln1 = pd.read_csv (FI